### ST001937 - AdaBoost

### AdaBoost e classificazione multiclasse
L'algoritmo **AdaBoost** è stato originariamente progettato per problemi di classificazione binaria, ma può essere esteso per gestire problemi di classificazione multiclasse mediante due tecniche di estensione, **One-vs-All** e **One-vs-One**:
- **One-vs-All (OvA)**: viene addestrato un classificatore binario per ogni classe, considerando una classe come positiva e tutte le altre come negative. Per classificare un'istanza, vengono eseguiti tutti i classificatori e l'istanza è assegnata alla classe del classificatore che restituisce la classificazione migliore.
- **One-vs-One (OvO)**: viene addestrato un classificatore binario per ciascuna coppia di classi possibili. Quando si classifica un'istanza, ogni classificatore restituisce un voto per la sua classe associata. L'istanza è assegnata alla classe con il maggior numero di voti.

### 1. Import Packages

In [1]:
# Librerie di base
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Librerie per la Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV

# Librerie per il Machine Learning
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as metrics
from sklearn.model_selection import KFold, train_test_split, cross_val_score

# Librerie per la Features Selection
from sklearn.feature_selection import SelectKBest, f_classif
# Librerie per la PCA
from sklearn.decomposition import PCA

print('All packages successfully loaded')

All packages successfully loaded


### 2. Load Data & Peak Sheet

In [2]:
df = pd.read_csv('../../../data/ST001937.csv')

# Rimuoviamo le colonne che non ci servono
df.drop(columns=["Sample ID", "RAW_FILE_NAME"], inplace=True)

# Visualizziamo le prime 5 righe del dataset
df.head()

,Pheotypes,"1,3,5(10)-estratrien-3,6- beta-17-beta-triol","1,5-anhydroglucitol",17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-1,17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-2,17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-3,17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-4,17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-5,17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-6,1-hexadecanol,...,tyrosine-1,tyrosine-2,urea-1,urea-2,urea-3,urea-4,uridine,valine,xanthine,xanthosine
0,Benign SPNS,0.437408,0.663764,0.700609,0.096924,0.173891,0.799124,0.573520,1.105910,0.677833,...,0.689488,0.037373,0.595031,0.234936,0.589201,0.127788,0.113960,0.216687,0.770860,0.174060
1,Benign SPNS,5.064922,0.653161,0.883121,0.056551,0.932342,0.394648,0.772451,0.623514,0.769287,...,0.537845,0.016517,0.484474,0.564398,0.482784,0.119055,0.009865,0.164879,0.456740,0.320723
2,Benign SPNS,1.046057,0.798040,0.838630,0.283674,0.119269,0.032103,0.807268,0.339667,1.104029,...,0.679699,1.079720,0.901623,0.559234,0.866856,0.159342,0.026922,0.252849,0.473077,0.158255
3,Benign SPNS,2.530700,1.318840,1.074717,0.431186,1.773602,0.627714,2.042268,0.913226,0.741269,...,0.837193,0.522064,1.433745,1.442062,1.406062,0.231101,0.278952,0.301997,0.940990,1.095022
4,Benign SPNS,0.027033,1.031146,0.863596,0.201422,0.222126,0.928862,1.004769,0.403102,0.790744,...,0.826921,0.462153,1.040359,1.154850,1.034635,0.198105,0.868354,0.302138,1.208439,0.557425


### 2.1 Data Cleaning

Notiamo che il dataset non presenta alcun nullo, di conseguenza non dobbiamo effettuare nessuna operazione di imputazione. Dobbiamo soltanto modificare i valori testuali della feature Pheotypes in valori numerici.

In [3]:
# Visualizziamo le colonne con dati mancanti
print(df.isnull().sum())
print("----------------------------------------------------")
print(df.isnull().sum().sum())

df['Pheotypes'] = df['Pheotypes'].apply(lambda x: 0 if x == 'Healthy Controls' else 1 if x == 'Benign SPNS' else 2)

Pheotypes                                        0
1,3,5(10)-estratrien-3,6- beta-17-beta-triol     0
1,5-anhydroglucitol                              0
17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-1    0
17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-2    0
                                                ..
urea-4                                           0
uridine                                          0
valine                                           0
xanthine                                         0
xanthosine                                       0
Length: 546, dtype: int64
----------------------------------------------------
0


### 3. Extract X & Y

In [4]:
X = df.drop(columns=['Pheotypes'])
X_features_names = X.columns
y = df.Pheotypes

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 4. Initial Model Build

In [5]:
# Definiamo il modello AdaBoost con gli iperparametri di default
model = AdaBoostClassifier()

# Addestriamo il modello
model.fit(X_train, y_train)

# Eseguiamo le previsioni sui dati di test
y_pred = model.predict(X_test)

### 5. Initial Model Evalutation

In [6]:
# Visualizziamo le metriche
print(f'Accuratezza: {metrics.accuracy_score(y_test, y_pred)}')
print(f'Precision: {metrics.precision_score(y_test, y_pred, average="macro")}')
print(f'Recall: {metrics.recall_score(y_test, y_pred, average="macro")}')
print(f'F1: {metrics.f1_score(y_test, y_pred, average="macro")}')
print("--------------------------------")
print(f'Matrice di confusione \n{metrics.confusion_matrix(y_test, y_pred)}')

Accuratezza: 0.8448275862068966
Precision: 0.7206349206349206
Recall: 0.6723463647368796
F1: 0.6730541296273111
--------------------------------
Matrice di confusione 
[[ 48   0   2]
 [  0   3  28]
 [  0   6 145]]


Le prestazioni iniziali sono ottime: abbiamo un'accuratezza di 84%. Proviamo con la K-Fold Cross Validation.

In [7]:
# Testiamo diverse configurazioni di K
max = 0
k_best = 0
model = AdaBoostClassifier()
for k in range(5, 11): 
    kfolds = KFold(n_splits=k, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, cv=kfolds)
    mean = np.mean(scores)
    if mean > max: 
        max = mean
        k_best = k
    print(f"K={k}, Accuratezza Media: {mean}")
    
print("-------------------------------------------")
print(f"K ottimale: {k_best}")
print(f"Accuratezza media ottimale: {max}")

K=5, Accuratezza Media: 0.810344827586207
K=6, Accuratezza Media: 0.8103288641988499
K=7, Accuratezza Media: 0.815573984248683
K=8, Accuratezza Media: 0.7956896551724137
K=9, Accuratezza Media: 0.809505544788975
K=10, Accuratezza Media: 0.7879310344827586
-------------------------------------------
K ottimale: 7
Accuratezza media ottimale: 0.815573984248683


Abbiamo ottenuto le prestazioni migliori con un numero di fold pari a 6.

In [8]:
# Creiamo l'oggetto K-Fold per la Cross-Validation con il numero di fold ottimale
kfolds = KFold(n_splits=k_best, shuffle=True, random_state=42)

### 6. Hyperparameters Optimization

L'**ottimizzazione degli iperparametri** è un passo fondamentale nello sviluppo di modelli predittivi robusti. Infatti, aderire ai parametri predefiniti impedisce ai modelli di raggiungere il massimo delle prestazioni. A tale scopo, utilizziamo la tecnica **Grid Search**.

In [9]:
# Creiamo un nuovo modello AdaBoost
model_2 = AdaBoostClassifier()

# Definiamo la griglia con i parametri da testare
param_grid = {
    'n_estimators': [50, 100, 150, 200, 250, 300],           
    'learning_rate': [0.01, 0.1, 1.0, 10],        
    'estimator': [None, DecisionTreeClassifier(max_depth=1)],
    'algorithm': ['SAMME', 'SAMME.R'],    
    'random_state': [42]                      
}

# Creiamo l'oggetto GridSearchCV
grid_search_model = GridSearchCV(model_2, param_grid, scoring='accuracy', cv=kfolds, n_jobs=-1)
grid_search_model.fit(X, y)

# Visualizziamo i risultati
best_params = grid_search_model.best_params_
print("Iperparametri migliori:", best_params)

KeyboardInterrupt: 

In [ ]:
# Utilizziamo gli iperparametri ottimizzati per creare un nuovo modello
best_model = AdaBoostClassifier(**best_params)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

In [ ]:
# Visualizziamo le metriche
print(f'Accuratezza: {metrics.accuracy_score(y_test, y_pred)}')
print(f'Precision: {metrics.precision_score(y_test, y_pred, average="macro", zero_division=1)}')
print(f'Recall: {metrics.recall_score(y_test, y_pred, average="macro")}')
print(f'F1: {metrics.f1_score(y_test, y_pred, average="macro")}')
print("--------------------------------")
print(f'Matrice di confusione \n{metrics.confusion_matrix(y_test, y_pred)}')

Accuratezza: 0.8577586206896551
Precision: 0.9402173913043478
Recall: 0.6533333333333333
F1: 0.6270281246827089
--------------------------------
Matrice di confusione 
[[ 48   0   2]
 [  0   0  31]
 [  0   0 151]]


Valutiamo le prestazioni con la K-Fold Cross Validation.

In [ ]:
# K-Fold Cross Validation
scores = cross_val_score(best_model, X, y, cv=kfolds, scoring='accuracy')
print(f'Accuratezza media: {scores.mean()}')

Accuratezza media: 0.8577895999582746


### 7. Feature Selection

Per la feature selection, al contrario di altri dataset già esaminati, non possiamo utilizzare il test del chi-quadro in quanto esso supporta solo dati non negativi. Per questo motivo, proviamo a utilizzare la tecnica *F-Classif* calcola il valore *F-statistic* e il *p-value* associato per ogni feature rispetto alla variabile target. L'*F-statistic* misura la differenza tra le medie dei gruppi rispetto alla varianza dei dati all'interno dei gruppi. Il *p-value* è quindi utilizzato per valutare la significatività statistica della differenza osservata.

In [ ]:
max_fs = 0 
k_fs = 0
best_features = []

for k in range(2, len(X.columns)):
    top_features = SelectKBest(score_func=f_classif, k=k).fit(X, y).get_support(indices=True)
    X_top = X.iloc[:, top_features]
    
    scores = cross_val_score(best_model, X_top, y, cv=kfolds, scoring='accuracy')
    mean = np.mean(scores)
    
    if mean > max_fs: 
        max_fs = mean
        k_fs = k
        best_features = X.columns[top_features].tolist()

    print(f"K={k}, Accuratezza Media: {mean}")

print(f"K ottimale: {k_fs}")
print(f"Accuratezza media ottimale: {max_fs}")
print(f"Features ottimali: {best_features}")

K=2, Accuratezza Media: 0.7223960778177645
K=3, Accuratezza Media: 0.761211078078548
K=4, Accuratezza Media: 0.8060710373963386
K=5, Accuratezza Media: 0.8207062014290931
K=6, Accuratezza Media: 0.8224378031606948
K=7, Accuratezza Media: 0.818140092838888
K=8, Accuratezza Media: 0.8198612632347572
K=9, Accuratezza Media: 0.8198612632347572
K=10, Accuratezza Media: 0.818140092838888
K=11, Accuratezza Media: 0.8345329369425755
K=12, Accuratezza Media: 0.8422834193918531
K=13, Accuratezza Media: 0.8414280498617848
K=14, Accuratezza Media: 0.8414228341939186
K=15, Accuratezza Media: 0.8396808011265843
K=16, Accuratezza Media: 0.8465863453815262
K=17, Accuratezza Media: 0.8457205445157253
K=18, Accuratezza Media: 0.8465759140457936
K=19, Accuratezza Media: 0.8534762426328691
K=20, Accuratezza Media: 0.8543420434986702
K=21, Accuratezza Media: 0.8560788608981379
K=22, Accuratezza Media: 0.8543524748344026
K=23, Accuratezza Media: 0.8543524748344026
K=24, Accuratezza Media: 0.8543524748344026

KeyboardInterrupt: 

L'accuratezza più alta si ha con le migliori 436 features, tuttavia è comunque minore dell'accuratezza su tutte e 544 le features. Di conseguenza, la feature selection non ci aiuta nelle nostre analisi.

In [ ]:
X_top = X[best_features]
scores = cross_val_score(best_model, X_top, y, cv=kfolds, scoring='accuracy')
print(f'Numero di features ottimali: {X_top.shape[1]}')
print(f'Accuratezza media: {scores.mean()}')

Numero di features ottimali: 467
Accuratezza media: 0.8595212016898763


### 8. PCA

In [ ]:
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.99) + 1
print("Numero di componenti principali:", d)

pca = PCA(n_components=d)
X_reduced = pca.fit_transform(X)

Numero di componenti principali: 86


In [ ]:
scores = cross_val_score(best_model, X_reduced, y, cv=kfolds, scoring='accuracy')
print(f'Accuratezza media: {scores.mean()}')

Accuratezza media: 0.7999928778733331


La PCA non ci aiuta nelle nostre analisi. Infatti, è impossibile arrivare a plottare le componenti principali in quanto saranno sempre maggiori di 3.

### 9. Results

Visualizziamo i risultati migliori.

In [ ]:
# Numero ottimale di fold 
kfolds = KFold(n_splits=k_best, shuffle=True, random_state=42)

# Modello eXtreme Gradient Boost con gli iperparametri ottimizzati
model = AdaBoostClassifier(**best_params)

# K-Fold Cross Validation del modello con gli iperparametri ottimizzati
scores = cross_val_score(model, X, y, cv=kfolds, scoring='accuracy')
print(f'Accuratezza media: {scores.mean()}')

# K-Fold Cross Validation del modello con gli iperparametri ottimizzati
scores = cross_val_score(model, X_top, y, cv=kfolds, scoring='accuracy')
print(f'Accuratezza media: {scores.mean()}')

Accuratezza media: 0.866371098409985
